In [ ]:
import math
from collections import deque
from typing import List, Tuple

# Problem 3.7
## Find the shortest path between two points on a plane with convex polygonal obstacles

1. Suppose the state space consists of all positions (x,y) in the plane. How many states are there? How many paths are there to the goal?

#### _Answer_
  - _The state space is defined as the set of nodes in a graph, represented as states, with their corresponding links being the actions that transition from one state to another._ 

    - _Given that the state space encompasses all (x, y) positions in the plane, and as a plane contains an infinite number of points, the state space itself contains an infinite number of points._ 

    - _**Since the state space is infinite, the number of states within it is also infinite**._

2. Explain briefly why the shortest path from one polygon vertex to any other in the scene must consist of straight-line segments joining some of the vertices of the polygons. Define a good state space now. How large is this state space? 

#### _Answer_
  - _Since the shortest path between two points in the presence of convex polygonal obstacles will always consist of straight-line segments between polygon vertices, the shortest path will always involve navigating as close to a straight line around the obstacles as possible. Therefore, an effective state space should include the vertices of the polygons, the start point, and the goal point._

3. Define the necessary functions to implement the search problem, including an function that takes a vertex as input and returns a set of vectors, each of which maps the current vertex to one of the vertices that can be reached in a straight line. (Do not forget the neighbors on the same polygon.) Use the straight-line distance for the heuristic function. 

In [4]:
class Coordinate:
    def __init__(self, x: int, y: int):
        self.position = (x, y)
        self.goal = (164, 79)
        self.predecessor = None

    def __repr__(self):
        return f"[{self.position[0]}, {self.position[1]}]"

    def __eq__(self, other):
        return self.position == other.position

    def __hash__(self):
        return hash(self.position)

    def is_goal(self) -> bool:
        return self.position == self.goal

    def dist_from_goal(self) -> float:
        return math.sqrt((self.goal[0] - self.position[0]) ** 2 + (self.goal[1] - self.position[1]) ** 2)

    def set_predecessor(self, predecessor):
        self.predecessor = predecessor

    def get_predecessor(self):
        return self.predecessor

class CoordinateSuccessorFunction:
    def __init__(self):
        self.coordinates = [
            Coordinate(16, 22), Coordinate(25, 28), Coordinate(25, 7), Coordinate(85, 28),
            Coordinate(85, 7), Coordinate(42, 83), Coordinate(24, 67), Coordinate(25, 45),
            Coordinate(45, 39), Coordinate(55, 62), Coordinate(69, 65), Coordinate(60, 34),
            Coordinate(77, 34), Coordinate(77, 81), Coordinate(91, 83), Coordinate(103, 70),
            Coordinate(80, 58), Coordinate(90, 45), Coordinate(95, 12), Coordinate(110, 26),
            Coordinate(103, 80), Coordinate(138, 80), Coordinate(103, 35), Coordinate(138, 35),
            Coordinate(125, 22), Coordinate(140, 29), Coordinate(153, 22), Coordinate(153, 6),
            Coordinate(140, 3), Coordinate(125, 6), Coordinate(145, 74), Coordinate(154, 80),
            Coordinate(159, 72), Coordinate(155, 29), Coordinate(164, 79)
        ]

    def get_successors(self, current_state: Coordinate) -> List[Coordinate]:
        successors = []
        for coord in self.coordinates:
            if self.are_neighbors(current_state, coord):
                successors.append(coord)
        return successors

    @staticmethod
    def are_neighbors(coord1: Coordinate, coord2: Coordinate) -> bool:
        return abs(coord1.position[0] - coord2.position[0]) <= 1 and abs(coord1.position[1] - coord2.position[1]) <= 1

def uniform_cost_search(start_state: Coordinate) -> List[Coordinate]:
    open_set = deque([start_state])
    closed_set = set()

    while open_set:
        current_state = open_set.popleft()
        if current_state.is_goal():
            print("Success: Goal found")
            print(current_state)
            return reconstruct_path(current_state)

        closed_set.add(current_state)

        successor_function = CoordinateSuccessorFunction()
        successors = successor_function.get_successors(current_state)

        for successor in successors:
            if successor not in closed_set and successor not in open_set:
                open_set.append(successor)
                successor.set_predecessor(current_state)

    print("Failed to Search")
    return []

def reconstruct_path(goal_state: Coordinate) -> List[Coordinate]:
    path = []
    current = goal_state
    while current is not None:
        path.append(current)
        current = current.get_predecessor()
    path.reverse()
    return path

def main():
    start_state = Coordinate(16, 22)
    path = uniform_cost_search(start_state)

    if path:
        print("Search path:")
        for coord in path:
            print(coord, end=" ")
        print(f"\nThe length of the path is: {sum(path[i].dist_from_goal() for i in range(len(path) - 1))}")

if __name__ == "__main__":
    main()

NameError: name 'List' is not defined

4. Apply one or more of the algorithms in this chapter to solve a range of problems in the domain, and comment on their performance.

# Problem 3.9
## Missionaries and Cannibals problem

1. Formulate the problem precisely, making only those distinctions necessary to ensure a valid solution. Draw a diagram of the complete state space. 

Initial State: Three missionaries, three cannibals, and the boat are on the left bank of the river.
 
Goal State: All three missionaries and three cannibals are on the right bank of the river.
 
Operators: The boat can carry one or two people from one bank to the other. The boat cannot move by itself.
 
Constraints: At no point should the number of cannibals on a specific bank outnumber the number of missionaries on that same bank. Otherwise, the cannibals will eat the missionaries.
 
State Space Diagram:
 
Explanation:
Here's a simplified representation of the state space:
 
Initial State: (3M, 3C, L)  --> (3M, 3C, R) : Goal State
 
L: Left Bank
R: Right Bank
 
Operators:
1. CC: Two cannibals cross from left to right.
2. C: One cannibal crosses from right to left.
3. MC: One missionary and one cannibal cross from left to right.
4. C: One cannibal crosses from right to left.
5. MC: One missionary and one cannibal cross from left to right.
6. C: One cannibal crosses from right to left.
7. CC: Two cannibals cross from left to right.
8. C: One cannibal crosses from right to left.
9. MC: One missionary and one cannibal cross from left to right.

2. Implement and solve the problem optimally using an appropriate search algorithm. Is it a good idea to check for repeated states? 

This problem can be optimally solved using a search algorithm such as Breadth-First Search (BFS) or Depth-First Search (DFS). BFS is often preferred because it guarantees the shortest path to the solution.
 
Checking for repeated states is essential to avoid infinite loops in the search process. Repeated states can occur when the same configuration of missionaries, cannibals, and boat positions is reached, thus creating an infinite back and forth.
 
To solve the Missionaries and Cannibals problem optimally, we'll use the Breadth-First Search (BFS) algorithm. BFS ensures that we find the shortest path to the goal state while avoiding any repeated states.
 
Here's a Python implementation:
 


In [5]:
from collections import deque

# Variables
m = 3
c = 3

# Define the initial state and goal state
goal_state = ((0, 0), (m, c), 1)  # (Missionaries on left, Cannibals on left, Boat position)
initial_state = ((m, c), (0, 0), 0)  # (Missionaries on left, Cannibals on left, Boat position)

# Define a function to check if a state is valid
def is_valid(state):
    left, right, boat = state
    if left[0] < 0 or left[1] < 0 or right[0] < 0 or right[1] < 0:
        return False
    if left[0] < left[1] and left[0] > 0:  # More cannibals than missionaries on the left bank
        return False
    if right[0] < right[1] and right[0] > 0:  # More cannibals than missionaries on the right bank
        return False
    return True

# Define a function to generate next possible states
def generate_next_states(state):
    left, right, boat = state
    possible_states = []

    # Define the possible moves
    moves = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]

    for move in moves:
        if boat == 0:
            new_left = (left[0] - move[0], left[1] - move[1])
            new_right = (right[0] + move[0], right[1] + move[1])
        else:
            new_left = (left[0] + move[0], left[1] + move[1])
            new_right = (right[0] - move[0], right[1] - move[1])

        new_state = (new_left, new_right, 1 - boat)
        if is_valid(new_state):
            possible_states.append(new_state)
    return possible_states

# Define the BFS function to find the solution
def bfs(initial_state, goal_state):
    visited = set()
    queue = deque([(initial_state, [])])

    while queue:
        state, path = queue.popleft()
        visited.add(tuple(state))  # Convert the state to a tuple before adding to set

        if state == goal_state:
            return path

        for next_state in generate_next_states(state):
            if tuple(next_state) not in visited:  # Convert the next_state to a tuple for comparison
                queue.append((next_state, path + [next_state]))

    return None

# Find and print the optimal solution
optimal_path = bfs(initial_state, goal_state)
if optimal_path:
    for step, state in enumerate(optimal_path):
        print(f"Step {step + 1}: {state}")
else:
    print("No solution found.")

Step 1: ((3, 1), (0, 2), 1)
Step 2: ((3, 2), (0, 1), 0)
Step 3: ((3, 0), (0, 3), 1)
Step 4: ((3, 1), (0, 2), 0)
Step 5: ((1, 1), (2, 2), 1)
Step 6: ((2, 2), (1, 1), 0)
Step 7: ((0, 2), (3, 1), 1)
Step 8: ((0, 3), (3, 0), 0)
Step 9: ((0, 1), (3, 2), 1)
Step 10: ((1, 1), (2, 2), 0)
Step 11: ((0, 0), (3, 3), 1)


This code uses BFS to find the optimal path from the initial state to the goal state while avoiding repeated states. It checks the validity of states, generates next possible states, and maintains a queue to explore the state space.
 
Checking for repeated states is crucial to avoid infinite loops, as the problem space can lead to revisiting the same states.
 
When you run the code, it will print the steps required to move the missionaries and cannibals from the initial state to the goal state while adhering to the rules of the puzzle.

3. Why do you think people have a hard time solving this puzzle, given that the state space is so simple? 

People often find this puzzle challenging because it involves a complex interplay of multiple constraints: maintaining the safety of the missionaries, preventing the cannibals from outnumbering the missionaries on either side and efficiently using the boat's capacity.
 
While the state space itself is simple to represent, the constraints and the need for a valid solution make it a non-trivial problem to solve manually. This puzzle requires careful planning and adherence to the defined rules to find a solution without violating any constraints.

In summary, the simplicity of the state space masks the complexity of the constraints and rules involved, making the missionaries and cannibals problem a classic puzzle in artificial intelligence and problem-solving.